# Firebase Uploader

In [1]:
import firebase_admin
from firebase_admin import credentials, storage, db
import pandas as pd
import os 
import dotenv

In [2]:
# Carregar variáveis de ambiente
dotenv.load_dotenv()

True

# Iniciar o Firebase

In [3]:
# Obter informações do json obtido no Firebase que foram adicionados no .env
service_account_info = {
    "type": os.getenv("FIREBASE_TYPE"),
    "project_id": os.getenv("FIREBASE_PROJECT_ID"),
    "private_key_id": os.getenv("FIREBASE_PRIVATE_KEY_ID"),
    "private_key": os.getenv("FIREBASE_PRIVATE_KEY"),
    "client_email": os.getenv("FIREBASE_CLIENT_EMAIL"),
    "client_id": os.getenv("FIREBASE_CLIENT_ID"),
    "auth_uri": os.getenv("FIREBASE_AUTH_URI"),
    "token_uri": os.getenv("FIREBASE_TOKEN_URI"),
    "auth_provider_x509_cert_url": os.getenv("FIREBASE_AUTH_PROVIDER_X509_CERT_URL"),
    "client_x509_cert_url": os.getenv("FIREBASE_CLIENT_X509_CERT_URL"),
    "universe_domain": os.getenv("FIREBASE_UNIVERSE_DOMAIN"),
}

In [4]:
# Criar credenciais
cred = credentials.Certificate(service_account_info)

# Iniciar app Firebase
firebase_admin.initialize_app(cred, {
    "storageBucket": "coffee-shop-app-c6116.firebasestorage.app",  # storage pras imagens
    "databaseURL": "https://coffee-shop-app-c6116-default-rtdb.firebaseio.com", # Realtime Database
})
# Obs: se der erro, precisa reiniciar o notebook para limpar as variáveis

# Firebase Storage

## Iniciar Bucket
Onde serão armazenadas as imagens no Firebase Storage.

In [5]:
bucket = storage.bucket()

## Upload dos dados

In [6]:
# Pasta de imagens
image_folder_path = "./products/images/"

In [7]:
# Tabela no banco de dados
products_collection = db.reference("products")

In [8]:
# Dataframe dos produtos
products_df = pd.read_json("products/products.json", lines=True)

products_df.head()

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,Latte.jpg
3,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",2.50,4.6,chocolat_biscotti.jpg
4,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],2.00,4.9,Espresso_shot.webp


In [9]:
# Função para fazer upload da imagem
def upload_image(bucket, image_path):
    # Nome da imagem: pegar o nome do arquivo (desconsiderando o caminho)
    image_name = image_path.split("/")[-1]
    # Criar o blob (objeto) no bucket
    blob = bucket.blob(f"product_images/{image_name}") # Caminho da imagem + nome da imagem

    # Upload image
    blob.upload_from_filename(image_path)

    # Tornar imagem pública e obter sua URL
    blob.make_public()
    return blob.public_url

In [10]:
for index, row in products_df.iterrows():
    # Obter imagem
    image_path = os.path.join(image_folder_path, row["image_path"])
    # URL da imagem no bucket
    image_url = upload_image(bucket, image_path)

    # Dados do produto
    product_data = row.to_dict() # converter df para dicionário
    product_data.pop("image_path")
    # Adicionar URL da imagem no bucket
    product_data["image_url"] = image_url

    # Adicionar ao Firebase
    products_collection.push().set(product_data)